In [1]:
import pandas as pd
import numpy as np
import os, sys
sys.path.insert(0, os.path.abspath('../..'))
import source.data_handling.data_preparation as dp
import numpy as np
import pandas as pd
pred_df = pd.read_csv("../../Data/ModelData/pred_df_loop.csv", index_col=0, parse_dates=True)


/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_10038/1609787396.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pred_df = pd.read_csv("../../Data/ModelData/pred_df_loop.csv", index_col=0, parse_dates=True)


In [2]:
import numpy as np
import pandas as pd

# ============================================================
# 1) Bull/Bear probability strategy backtest
#    w_t = clip(1 - P_t(bear), 0, 1)
# ============================================================
def backtest_prob_regime_strategy(
    pred_df: pd.DataFrame,
    price_col: str,
    prob_bear_col: str = "pred_prob",   # P(bear) known at t for next day
    rf_col: str | None = None,          # optional column in pred_df for risk-free total return per period
    rf_const: float = 0.0,              # used if rf_col is None
    tc_bps: float = 0.0,                # transaction cost per 100% turnover in bps
    ts_col: str = "timestamp",
):
    """
    Assumes: signal at time t is used for return from t->t+1.

    Equity return (fwd): price.pct_change().shift(-1)
    Risk-free return (fwd): rf.shift(-1) or constant

    Strategy net return (fwd):
      r_{t+1} = w_t * r_eq_{t+1} + (1-w_t) * r_rf_{t+1} - tc * |w_t - w_{t-1}|
    """

    df = pred_df.copy()

    # robust indexing
    if ts_col in df.columns:
        df[ts_col] = pd.to_datetime(df[ts_col])
        df = df.sort_values(ts_col).set_index(ts_col)
    else:
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()

    df = df.dropna(subset=[price_col, prob_bear_col]).copy()
    if df.empty:
        raise ValueError("No valid rows after dropping NaNs for price/probability columns.")

    price = pd.to_numeric(df[price_col], errors="coerce")
    p_bear = pd.to_numeric(df[prob_bear_col], errors="coerce").clip(0.0, 1.0)

    # forward equity return (t -> t+1)
    r_eq_fwd = price.pct_change().shift(-1)

    # forward risk-free return
    if rf_col is not None and rf_col in df.columns:
        rf = pd.to_numeric(df[rf_col], errors="coerce").fillna(rf_const)
    else:
        rf = pd.Series(rf_const, index=df.index, dtype=float)
    r_rf_fwd = rf.shift(-1)

    # model weight (your rule)
    w = (1.0 - p_bear).clip(0.0, 1.0)

    # turnover + costs (paid at t)
    turnover = w.diff().abs().fillna(0.0)
    cost = (tc_bps / 10000.0) * turnover

    # strategy returns realized next day
    strat_gross = w * r_eq_fwd + (1.0 - w) * r_rf_fwd
    strat_net = strat_gross - cost

    out = pd.DataFrame(index=df.index)
    out["p_bear"] = p_bear
    out["w"] = w
    out["turnover"] = turnover
    out["cost"] = cost
    out["r_eq_fwd"] = r_eq_fwd
    out["r_rf_fwd"] = r_rf_fwd
    out["model_gross"] = strat_gross
    out["model_net"] = strat_net

    # baselines (next-day returns)
    out["w100_net"] = r_eq_fwd
    out["w50_net"] = 0.5 * r_eq_fwd + 0.5 * r_rf_fwd
    out["w0_net"] = r_rf_fwd

    # optional: hard-threshold timing benchmark (very common)
    # invest if P(bear) < 0.5
    w_thr = (p_bear < 0.5).astype(float)
    out["thr_w"] = w_thr
    out["thr_turnover"] = w_thr.diff().abs().fillna(0.0)
    out["thr_cost"] = (tc_bps / 10000.0) * out["thr_turnover"]
    out["thr_net"] = (w_thr * r_eq_fwd + (1.0 - w_thr) * r_rf_fwd) - out["thr_cost"]

    # drop last row (no forward return)
    out = out.dropna(subset=["r_eq_fwd", "r_rf_fwd"])
    return out


# ============================================================
# 2) More metrics (works for equity premium AND this regime strategy)
# ============================================================
def perf_stats_plus(
    total_returns: pd.Series,
    rf_returns: pd.Series | None = None,
    benchmark_returns: pd.Series | None = None,
    periods_per_year: int = 12,
    weights: pd.Series | None = None,
    turnover: pd.Series | None = None,
    gamma_utility: float | None = None,
    var_level: float = 0.05
):
    """
    Returns a dict of richer performance stats.
    total_returns: portfolio total return per period (e.g., monthly, daily)
    rf_returns: risk-free total return per period (optional; for excess, Sharpe, alpha/beta)
    benchmark_returns: benchmark total returns per period (optional; for IR, alpha/beta)
    weights/turnover: optional series for avg weight / turnover stats
    gamma_utility: if provided, computes annualized mean-variance utility: ppY * (mean - gamma/2*var)
    """

    r = total_returns.dropna().astype(float)
    if len(r) < 3:
        return {}

    wealth = (1.0 + r).cumprod()
    total_ret = float(wealth.iloc[-1] - 1.0)
    cagr = float(wealth.iloc[-1] ** (periods_per_year / len(r)) - 1.0)
    ann_vol = float(r.std(ddof=0) * np.sqrt(periods_per_year))

    # excess returns for Sharpe/Sortino/alpha
    if rf_returns is not None:
        rf = rf_returns.reindex(r.index).astype(float)
        rex = (r - rf).dropna()
    else:
        rex = r.copy()

    sharpe = np.nan
    if rex.std(ddof=0) > 0:
        sharpe = float((rex.mean() / rex.std(ddof=0)) * np.sqrt(periods_per_year))

    # Sortino (downside deviation on excess)
    downside = rex[rex < 0]
    sortino = np.nan
    if len(rex) > 1:
        dd = np.sqrt((downside ** 2).mean()) if len(downside) > 0 else 0.0
        if dd > 0:
            sortino = float((rex.mean() / dd) * np.sqrt(periods_per_year))

    # drawdown + max drawdown duration
    peak = wealth.cummax()
    dd = wealth / peak - 1.0
    max_dd = float(dd.min())

    below_peak = wealth < peak
    dd_dur = 0
    cur = 0
    for b in below_peak:
        cur = cur + 1 if b else 0
        dd_dur = max(dd_dur, cur)

    calmar = np.nan
    if max_dd < 0:
        calmar = float(cagr / abs(max_dd))

    # hit rate + tail stats
    hit_rate = float((r > 0).mean())
    skew = float(r.skew())
    kurt = float(r.kurt())

    # VaR / CVaR (on total returns)
    var_q = float(np.quantile(r, var_level))
    cvar = float(r[r <= var_q].mean()) if (r <= var_q).any() else np.nan

    # Information ratio vs benchmark (on active returns)
    ir = np.nan
    if benchmark_returns is not None:
        b = benchmark_returns.reindex(r.index).astype(float)
        active = (r - b).dropna()
        if active.std(ddof=0) > 0:
            ir = float((active.mean() / active.std(ddof=0)) * np.sqrt(periods_per_year))

    # alpha/beta vs benchmark (using excess returns if rf is available)
    alpha_ann = np.nan
    beta = np.nan
    if benchmark_returns is not None:
        b = benchmark_returns.reindex(r.index).astype(float)
        if rf_returns is not None:
            rf = rf_returns.reindex(r.index).astype(float)
            y = (r - rf).dropna()
            x = (b - rf).reindex(y.index).dropna()
            y = y.reindex(x.index)
        else:
            y = r.dropna()
            x = b.reindex(y.index).dropna()
            y = y.reindex(x.index)

        if len(x) > 5 and np.var(x.values) > 0:
            cov = np.cov(x.values, y.values, ddof=0)[0, 1]
            beta = float(cov / np.var(x.values))
            alpha = float(y.mean() - beta * x.mean())
            alpha_ann = float(alpha * periods_per_year)

    # turnover / weights summaries
    avg_w = np.nan
    avg_turnover = np.nan
    if weights is not None:
        avg_w = float(pd.Series(weights).dropna().mean())
    if turnover is not None:
        avg_turnover = float(pd.Series(turnover).dropna().mean())

    # annualized utility (mean-variance)
    ann_u = np.nan
    if gamma_utility is not None:
        ann_u = float(periods_per_year * (r.mean() - (gamma_utility / 2.0) * r.var(ddof=0)))

    return {
        "TotalReturn": total_ret,
        "CAGR": cagr,
        "AnnVol": ann_vol,
        "Sharpe(excess)": sharpe,
        "Sortino(excess)": sortino,
        "MaxDrawdown": max_dd,
        "MaxDDDuration": float(dd_dur),
        "Calmar": calmar,
        "HitRate": hit_rate,
        "Skew": skew,
        "Kurt": kurt,
        "VaR(5%)": var_q,
        "CVaR(5%)": cvar,
        "IR(vs bm)": ir,
        "Alpha_ann": alpha_ann,
        "Beta": beta,
        "AvgWeight": avg_w,
        "AvgTurnover": avg_turnover,
        "AnnUtility": ann_u,
        "N": float(len(r)),
    }


def compare_strategies_plus(
    R: pd.DataFrame,                 # total returns per strategy (columns)
    rf: pd.Series | None = None,     # risk-free total return
    benchmark_col: str | None = None,
    periods_per_year: int = 12,
    weights: dict[str, pd.Series] | None = None,
    turnover: dict[str, pd.Series] | None = None,
    gamma_utility: float | None = None,
):
    """
    R: DataFrame of total returns for each strategy, aligned by index
    rf: optional risk-free series aligned to R
    benchmark_col: which column in R to treat as benchmark for IR/alpha/beta (e.g. "W100")
    weights/turnover: optional dicts keyed by strategy name
    """
    R = R.copy().dropna(how="any")

    bm = R[benchmark_col] if benchmark_col is not None and benchmark_col in R.columns else None

    rows = []
    for col in R.columns:
        w = weights.get(col) if weights is not None else None
        to = turnover.get(col) if turnover is not None else None

        stats = perf_stats_plus(
            total_returns=R[col],
            rf_returns=rf.reindex(R.index) if rf is not None else None,
            benchmark_returns=bm if (bm is not None and col != benchmark_col) else None,
            periods_per_year=periods_per_year,
            weights=w.reindex(R.index) if isinstance(w, pd.Series) else w,
            turnover=to.reindex(R.index) if isinstance(to, pd.Series) else to,
            gamma_utility=gamma_utility,
        )
        stats["Strategy"] = col
        rows.append(stats)

    out = pd.DataFrame(rows).set_index("Strategy")

    # If utility is present and you want deltas vs benchmark:
    if gamma_utility is not None and benchmark_col is not None and benchmark_col in out.index:
        out[f"Δu vs {benchmark_col}"] = np.nan
        for col in out.index:
            if col == benchmark_col:
                continue
            out.loc[col, f"Δu vs {benchmark_col}"] = out.loc[col, "AnnUtility"] - out.loc[benchmark_col, "AnnUtility"]

    return out


In [3]:
bt = backtest_prob_regime_strategy(
    pred_df=pred_df,
    price_col="M1WO_O",
    prob_bear_col="pred_prob",
    rf_col=None,          # or "Rfree" if you have it
    rf_const=0.0,
    tc_bps=5.0
)

R = pd.DataFrame({
    "Model": bt["model_net"],
    "W100": bt["w100_net"],   # buy & hold equity
    "W50":  bt["w50_net"],
    "Cash": bt["w0_net"],
    "Thr":  bt["thr_net"],    # optional timing baseline
}, index=bt.index).dropna()

weights = {
    "Model": bt["w"],
    "Thr": bt["thr_w"],
    "W100": pd.Series(1.0, index=bt.index),
    "W50": pd.Series(0.5, index=bt.index),
    "Cash": pd.Series(0.0, index=bt.index),
}
turnover = {
    "Model": bt["turnover"],
    "Thr": bt["thr_turnover"],
    "W100": pd.Series(0.0, index=bt.index),
    "W50": pd.Series(0.0, index=bt.index),
    "Cash": pd.Series(0.0, index=bt.index),
}

summary = compare_strategies_plus(
    R=R,
    rf=None,                    # or bt["r_rf_fwd"] if you want excess-based Sharpe
    benchmark_col="W100",
    periods_per_year=252,       # daily
    weights=weights,
    turnover=turnover,
    gamma_utility=5.0
)
print(summary)


          TotalReturn      CAGR    AnnVol  Sharpe(excess)  Sortino(excess)  \
Strategy                                                                     
Model        1.356571  0.044940  0.081352        0.581170         0.536249   
W100         3.214231  0.076557  0.166858        0.525909         0.487800   
W50          1.197987  0.041213  0.083429        0.525909         0.487800   
Cash         0.000000  0.000000  0.000000             NaN              NaN   
Thr          1.478974  0.047657  0.092038        0.552005         0.448598   

          MaxDrawdown  MaxDDDuration    Calmar   HitRate      Skew  ...  \
Strategy                                                            ...   
Model       -0.239013         1494.0  0.188023  0.543346 -0.651737  ...   
W100        -0.578172         1438.0  0.132411  0.547009 -0.482153  ...   
W50         -0.339600         1366.0  0.121358  0.547009 -0.482153  ...   
Cash         0.000000            0.0       NaN  0.000000  0.000000  ...   
Thr

In [4]:
bt

,p_bear,w,turnover,cost,r_eq_fwd,r_rf_fwd,model_gross,model_net,w100_net,w50_net,w0_net,thr_w,thr_turnover,thr_cost,thr_net
timestamp,,,,,,,,,,,,,,,
2007-01-01,0.000629,0.999371,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000
2007-01-02,0.000682,0.999318,0.000053,2.641723e-08,0.007270,0.0,0.007265,0.007265,0.007270,0.003635,0.0,1.0,0.0,0.0,0.007270
2007-01-03,0.000492,0.999508,0.000189,9.472939e-08,-0.003741,0.0,-0.003740,-0.003740,-0.003741,-0.001871,0.0,1.0,0.0,0.0,-0.003741
2007-01-04,0.000606,0.999394,0.000114,5.698577e-08,-0.002872,0.0,-0.002870,-0.002870,-0.002872,-0.001436,0.0,1.0,0.0,0.0,-0.002872
2007-01-05,0.000537,0.999463,0.000069,3.447625e-08,-0.009803,0.0,-0.009798,-0.009798,-0.009803,-0.004902,0.0,1.0,0.0,0.0,-0.009803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-24,0.103272,0.896728,0.093715,4.685731e-05,0.006401,0.0,0.005740,0.005693,0.006401,0.003201,0.0,1.0,0.0,0.0,0.006401
2025-10-27,0.075741,0.924259,0.027531,1.376554e-05,0.010863,0.0,0.010041,0.010027,0.010863,0.005432,0.0,1.0,0.0,0.0,0.010863
2025-10-28,0.057736,0.942264,0.018005,9.002511e-06,0.001686,0.0,0.001589,0.001580,0.001686,0.000843,0.0,1.0,0.0,0.0,0.001686
